In [10]:
import os
import pandas as pd
import glob

### Load Data from UCI Machine Learning Repository:

In [35]:
# only have to run once

try:
    # Create data Directory
    os.mkdir('data')
    print("data Directory Created ") 
except FileExistsError:
    print("data Directory already exists")


!wget -O data/processed.cleveland.data.csv http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data
!wget -O data/processed.hungarian.data.data.csv http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/reprocessed.hungarian.data
!wget -O data/processed.switzerland.data.csv http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.switzerland.data
!wget -O data/processed.va.data.csv http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.va.data
!wget -O data/heart-disease.names.csv http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/heart-disease.names
                

data Directory Created 


--2020-01-09 19:50:14--  http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18461 (18K) [application/x-httpd-php]
Saving to: 'data/processed.cleveland.data.csv'

     0K .......... ........                                   100% 87.0K=0.2s

2020-01-09 19:50:15 (87.0 KB/s) - 'data/processed.cleveland.data.csv' saved [18461/18461]

--2020-01-09 19:50:15--  http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/reprocessed.hungarian.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11058 (11K) [application/x-httpd-php]
Saving to: 'data/processed.hungari

### Concatenate 4 datasets to one:

In [36]:
allFiles = glob.glob('data' + '/*.data.csv')

heart_disease_df = pd.DataFrame()
list_ = []

col_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']

for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, names=col_names)
    list_.append(df)
heart_disease_df = pd.concat(list_, sort=False, ignore_index=True)

In [37]:
print(heart_disease_df)

    age  sex   cp trestbps chol fbs restecg thalach exang oldpeak slope   ca  \
0    63  1.0  1.0      145  233   1       2     150     0     2.3     3  0.0   
1    67  1.0  4.0      160  286   0       2     108     1     1.5     2  3.0   
2    67  1.0  4.0      120  229   0       2     129     1     2.6     2  2.0   
3    37  1.0  3.0      130  250   0       0     187     0     3.5     3  0.0   
4    41  0.0  2.0      130  204   0       2     172     0     1.4     1  0.0   
..   ..  ...  ...      ...  ...  ..     ...     ...   ...     ...   ...  ...   
915  54  0.0  4.0      127  333   1       1     154     0       0     ?    ?   
916  62  1.0  1.0        ?  139   0       1       ?     ?       ?     ?    ?   
917  55  1.0  4.0      122  223   1       1     100     0       0     ?    ?   
918  58  1.0  4.0        ?  385   1       2       ?     ?       ?     ?    ?   
919  62  1.0  2.0      120  254   0       2      93     1       0     ?    ?   

    thal  num  
0    6.0  0.0  
1    3.